In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error

In [2]:

train_data=pd.read_csv("/home/jupyter/kaggle-notebooks/Dataset/train.csv")
X=train_data.drop(columns=["id","Hardness"])
y=train_data["Hardness"]
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)


## Random Forest Regressor

In [3]:
from sklearn.ensemble import RandomForestRegressor
RF_regressor= RandomForestRegressor()
RF_regressor.fit(X_train,y_train)
y_pred=RF_regressor.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae



0.6749999999999998

## HistGradientBoosting Regressor

In [4]:
from sklearn.ensemble import HistGradientBoostingRegressor
HGBR=HistGradientBoostingRegressor()
HGBR.fit(X_train,y_train)
y_pred=HGBR.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.6381275442355352

## SVM Regressor

In [5]:
from sklearn import svm
svr = svm.SVR()
svr.fit(X_train,y_train)
y_pred=svr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.9019036394186755

## XGB Regressor

In [6]:
!pip install xgboost

In [7]:
import xgboost as xgb
xgbr=xgb.XGBRegressor()
xgbr.fit(X_train,y_train)
y_pred=xgbr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.6781597137451172

## Catboost Regressor

In [8]:
from catboost import CatBoostRegressor
cbr=CatBoostRegressor()
cbr.fit(X_train,y_train)
y_pred=cbr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae


Learning rate set to 0.057227
0:	learn: 1.6577228	total: 59.3ms	remaining: 59.2s
1:	learn: 1.6298259	total: 62.2ms	remaining: 31s
2:	learn: 1.6028105	total: 64.9ms	remaining: 21.6s
3:	learn: 1.5778444	total: 67.7ms	remaining: 16.9s
4:	learn: 1.5552126	total: 70.7ms	remaining: 14.1s
5:	learn: 1.5346130	total: 73.7ms	remaining: 12.2s
6:	learn: 1.5151060	total: 76.5ms	remaining: 10.8s
7:	learn: 1.4969527	total: 79.3ms	remaining: 9.83s
8:	learn: 1.4809247	total: 82.1ms	remaining: 9.04s
9:	learn: 1.4661157	total: 85ms	remaining: 8.42s
10:	learn: 1.4519091	total: 87.9ms	remaining: 7.9s
11:	learn: 1.4386087	total: 90.6ms	remaining: 7.46s
12:	learn: 1.4285161	total: 93.6ms	remaining: 7.11s
13:	learn: 1.4174654	total: 96.5ms	remaining: 6.8s
14:	learn: 1.4068424	total: 99.4ms	remaining: 6.52s
15:	learn: 1.3970505	total: 102ms	remaining: 6.29s
16:	learn: 1.3889331	total: 105ms	remaining: 6.07s
17:	learn: 1.3809595	total: 108ms	remaining: 5.88s
18:	learn: 1.3733801	total: 111ms	remaining: 5.74s
19

0.6620364074719269

## LGBM Regressor


In [9]:
from lightgbm import LGBMRegressor
lgbmr=LGBMRegressor()
lgbmr.fit(X_train,y_train)
y_pred=lgbmr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002615 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2462
[LightGBM] [Info] Number of data points in the train set: 8325, number of used features: 11
[LightGBM] [Info] Start training from score 4.650150


0.6508308930496491

## Gradient Boosting Regressor

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
gbr=GradientBoostingRegressor()
gbr.fit(X_train,y_train)
y_pred=gbr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.69677391981787

## ExtraTree Regressor

In [11]:
from sklearn.tree import ExtraTreeRegressor
etr=ExtraTreeRegressor()
etr.fit(X_train,y_train)
y_pred=etr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.7999999999999998

## Decision Tree Regressor

In [12]:
from sklearn.tree import DecisionTreeRegressor
dcr=DecisionTreeRegressor()
dcr.fit(X_train,y_train)
y_pred=dcr.predict(X_test)
mae=median_absolute_error(y_pred,y_test)
mae

0.7999999999999998

Lets try optuna

In [13]:
!pip install optuna

In [14]:
# import optuna
# from sklearn.model_selection import cross_val_score
# # define objective function for optuna
# def objective(trial):
#     # Define the hyperparameters to tune
#     params = {
#         'objective': 'reg:squarederror',
#         'eval_metric': 'mae',
#         'max_depth': trial.suggest_int('max_depth', 2, 20),
#         'learning_rate': trial.suggest_float('learning_rate', 0.002, 0.2,log=True),
#         'subsample': trial.suggest_float('subsample', 0.5, 1),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
#         'n_estimators': trial.suggest_int('n_estimators', 50, 1200, step=50),
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10,log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10,log=True),
#         'gamma': trial.suggest_float('gamma', 1e-5, 10,log=True),
#         'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
#     }

#     # Train an XGB Regressor model with the given hyperparameters
#     model = xgb.XGBRegressor(**params,random_state=42)
#     scores=cross_val_score(model,X,y,cv=5)
#     return scores.mean()


# # Create an Optuna study object and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# # Print the best hyperparameters and score found by Optuna
# print(f'Best score: {study.best_value:.4f}')
# print(f'Best params: {study.best_params}')

In [15]:
best_par=study.best_params
model=xgb.XGBRegressor(**best_par,random_state=42)
model.fit(X,y)

NameError: name 'study' is not defined

In [16]:
test_dataset=pd.read_csv("/home/jupyter/kaggle-notebooks/Dataset/test.csv")
sample_submission = pd.read_csv('/home/jupyter/kaggle-notebooks/Dataset/sample_submission.csv')

In [ ]:
sample_submission=test_dataset[["id","Hardness"]]
sample_submission.to_csv("Mysubmission1R.csv",index=False)

In [ ]:
# def objective(trial):
#     # Define the hyperparameters to tune
#     params = {
#         'max_iter': trial.suggest_int('max_iter', 100, 2000, step=100),
#         'loss':'absolute_error',
#         'max_leaf_nodes': trial.suggest_int('max_leaf_nodes',10,50),
#         'learning_rate': trial.suggest_float('learning_rate', 0.003, 0.2,log=True),
#         'max_depth': trial.suggest_int('max_depth', 3, 30),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
#         'l2_regularization': trial.suggest_float('l2_regularization', 1e-10, 1.0,log=True)
#     }

#     # Train an XGB Regressor model with the given hyperparameters
#     model = HistGradientBoostingRegressor(**params)
#     scores=cross_val_score(model,X,y,cv=5,scoring='neg_median_absolute_error')
#     return scores.mean()


# # Create an Optuna study object and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=200)

# # Print the best hyperparameters and score found by Optuna
# print(f'Best score: {study.best_value:.4f}')
# print(f'Best params: {study.best_params}')

Best score: -0.5180
Best params: {'max_iter': 1200, 'max_leaf_nodes': 50, 'learning_rate': 0.0131995921627786, 'max_depth': 17, 'min_samples_leaf': 10, 'l2_regularization': 1.508629493481232e-08}

In [17]:
params={'max_iter': 1200, 
        'max_leaf_nodes': 50, 
        'learning_rate': 0.0131995921627786, 
        'max_depth': 17, 
        'min_samples_leaf': 10, 
        'l2_regularization': 1.508629493481232e-08}

model = HistGradientBoostingRegressor(**params)
model.fit(X,y)


HistGradientBoostingRegressor(l2_regularization=1.508629493481232e-08,
                              learning_rate=0.0131995921627786, max_depth=17,
                              max_iter=1200, max_leaf_nodes=50,
                              min_samples_leaf=10)

In [18]:
test_dataset["Hardness"]=model.predict(test_dataset.drop(columns='id'))

sample_submission=test_dataset[["id","Hardness"]]
sample_submission.to_csv("Mysubmission2.csv",index=False)